This Query is created by user undefined

In [2]:
%% sql
```
-- Pre - Processing step
WITH PREPROCESSING AS (
  SELECT 'Hospitalization' AS CLAIM_CATEGORY
  FROM DUAL
),

-- Business Requirement
AGG_CLAIM_STATUS_DAILY AS (
  SELECT INTIMATION_NUMBER,
         PATIENT_NAME,
         HEALTH_ID_CARD_NUMBER AS MEMBER_CODE,
         PATIENT_AGE,
         CASE GENDER_ID
           WHEN 111 THEN 'Male'
           WHEN 112 THEN 'Female'
           ELSE 'Others'
         END AS GENDER,
         POLICY_NUMBER,
         PROPOSER_REG_MOBILE_NUMBER AS PHONE_NUMBER,
         CASE
           WHEN DATALENGTH(ATTENDER_MOBILE_NUMBER) <> 0 THEN CALLER_MOBILE_NUMBER + ',' + ATTENDER_MOBILE_NUMBER
           ELSE CALLER_MOBILE_NUMBER
         END AS ALTERNATE_PHONE_NUMBER,
         HOSPITAL_NAME,
         DIAGNOSIS,
         ADMISSION_DATE AS DOA,
         DISCHARGE_DATE AS DOD,
         TYPE_OF_CLAIM,
         NVL(SUM(NVL(CLAIMED_AMOUNT_BILLS, 0)), 0) AS ESTIMATE_AMT,
         APPROVED_AMOUNT AS APPROVED_AMOUNT
  FROM AGG_CLAIM_STATUS_DAILY
  LEFT JOIN IMS_CLS_CLAIM_AMOUNT_DETAILS ON AMT.CASHLESS_KEY = CLS.CASHLESS_KEY
  WHERE NVL(DIAG.DELETED_FLAG, 1) = 1
    AND NVL(DIAC.DELETED_FLAG, 1) = 1
    AND NVL(PRO.DELETED_FLAG, 1) = 1
  GROUP BY CASHLESS_KEY
),

IMS_CLS_POLICY_INSURED AS (
  SELECT GENDER_ID, INSURED_KEY
  FROM IMS_CLS_POLICY_INSURED
),

IMS_CLS_INTIMATION AS (
  SELECT INT.INTIMATION_KEY, AGG.INTIMATION_KEY AS AGG_INTIMATION_KEY
  FROM IMS_CLS_INTIMATION
  LEFT JOIN AGG_CLAIM_STATUS_DAILY AGG ON INT.INTIMATION_KEY = AGG.INTIMATION_KEY
),

MAS_HOSPITALS AS (
  SELECT HOSPITAL_IRDA_CODE AS HOSPITAL_CODE, CITY, STATE, HOSPITAL_KEY
  FROM MAS_HOSPITALS
),

MAS_ICD_CHAPTER AS (
  SELECT VALUE, ICD_CHAPTER_KEY
  FROM MAS_ICD_CHAPTER
),

MAS_ICD_CODE AS (
  SELECT VALUE, ICD_CODE_KEY
  FROM MAS_ICD_CODE
),

IMS_CLS_DIAGNOSIS AS (
  SELECT DIAGNOSIS_ID, TRANSACTION_KEY
  FROM IMS_CLS_DIAGNOSIS
  WHERE NVL(DELETED_FLAG, 1) = 1
),

IMS_CLS_PROCEDURE AS (
  SELECT PROCEDURE_NAME, TRANSACTION_KEY
  FROM IMS_CLS_PROCEDURE
  WHERE NVL(DELETED_FLAG, 1) = 1
),

IMS_CLS_CASHLESS AS (
  SELECT CLS.STATUS_ID, CLS.ITERATION_NUMBER, ROD_CLG_NUMBER
  FROM IMS_CLS_CASHLESS CLS
),

-- SQL Expression
SELECT AGG_CLAIM_STATUS_DAILY.INTIMATION_NUMBER AS CLAIM_NO,
       AGG_CLAIM_STATUS_DAILY.PATIENT_NAME AS PATIENT_NAME,
       AGG_CLAIM_STATUS_DAILY.HEALTH_ID_CARD_NUMBER AS MEMBER_CODE,
       AGG_CLAIM_STATUS_DAILY.PATIENT_AGE AS PATIENT_AGE,
       AGG_CLAIM_STATUS_DAILY.GENDER AS GENDER,
       AGG_CLAIM_STATUS_DAILY.POLICY_NUMBER AS POLICY_NO,
       AGG_CLAIM_STATUS_DAILY.PROPOSER_REG_MOBILE_NUMBER AS PHONE_NUMBER,
       AGG_CLAIM_STATUS_DAILY.CALLER_MOBILE_NUMBER AS ALTERNATE_PHONE_NUMBER,
       AGG_CLAIM_STATUS_DAILY.HOSPITAL_NAME AS HOSPITAL_NAME,
       MAS_HOSPITALS.CITY AS HOSPITAL_CITY,
       MAS_HOSPITALS.STATE AS HOSPITAL_STATE,
       AGG_CLAIM_STATUS_DAILY.DIAGNOSIS AS DIAGNOSIS,
       MAS_ICD_CHAPTER.VALUE AS ICD_CHAPTER,
       MAS_ICD_CODE.VALUE AS ICD_LEVEL_1,
       IMS_CLS_PROCEDURE.PROCEDURE_NAME AS PROCEDURE_DESCRIPTION,
       AGG_CLAIM_STATUS_DAILY.ADMISSION_DATE AS DOA,
       AGG_CLAIM_STATUS_DAILY.DISCHARGE_DATE AS DOD,
       AGG_CLAIM_STATUS_DAILY.TYPE_OF_CLAIM AS CLAIM_TYPE,
       PREPROCESSING.CLAIM_CATEGORY AS CLAIM_CATEGORY,
       AGG_CLAIM_STATUS_DAILY.APPROVED_AMOUNT AS APPROVED_AMOUNT,
       CASE
         WHEN IMS_CLS_CASHLESS.STATUS_ID IN (22, 30, 38, 38, 63, 23, 31) THEN 'Approved'
         ELSE 'Others'
       END AS CLAIM_STATUS
FROM AGG_CLAIM_STATUS_DAILY
LEFT JOIN IMS_CLS_POLICY_INSURED INS ON INS.INSURED_KEY = AGG_CLAIM_STATUS_DAILY.INSURED_KEY
LEFT JOIN IMS_CLS_INTIMATION INT ON INT.INTIMATION_KEY = AGG_CLAIM_STATUS_DAILY.INTIMATION_KEY
LEFT JOIN MAS_HOSPITALS MAS ON MAS.HOSPITAL_KEY = INT.HOSPITAL_ID
LEFT JOIN IMS_CLS_DIAGNOSIS DIAG ON DIAG.TRANSACTION_KEY = AGG_CLAIM_STATUS_DAILY.CASHLESS_KEY AND DIAG.PRIMARY_DIAGNOSIS = 'Y'
LEFT JOIN MAS_ICD_CHAPTER CHA ON CHA.ICD_CHAPTER_KEY = DIAG.ICD_CHAPTER_ID
LEFT JOIN IMS_CLS_DIAGNOSIS DIAC ON DIAC.TRANSACTION_KEY = AGG_CLAIM_STATUS_DAILY.CASHLESS_KEY
LEFT JOIN MAS_ICD_CODE COD ON COD.ICD_CODE_KEY = DIAC.ICD_CODE_ID
LEFT JOIN IMS_CLS_PROCEDURE PRO ON PRO.TRANSACTION_KEY = AGG_CLAIM_STATUS_DAILY.CASHLESS_KEY
LEFT JOIN IMS_CLS_CASHLESS CLS ON CLS.ITERATION_NUMBER = AGG_CLAIM_STATUS_DAILY.ROD_CLG_NUMBER
LEFT JOIN PREPROCESSING ON 1 = 1
```